In [245]:
import pandas as pd
import numpy as np
import missingno as msno
import matplotlib.pyplot as plt
import seaborn as sns
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import GradientBoostingClassifier,RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,ExtraTreesClassifier,StackingClassifier, HistGradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score,classification_report, roc_auc_score
from sklearn.preprocessing import LabelEncoder, KBinsDiscretizer, RobustScaler
from sklearn.model_selection import KFold,StratifiedKFold, GroupKFold,train_test_split
import gc
import datetime
from tqdm.notebook import tqdm ,tnrange
import warnings
warnings.filterwarnings("ignore")
np.random.seed(0)
pd.set_option('display.max_columns', 500)

In [246]:
df_train = pd.read_csv("train_Df64byy.csv")
df_test = pd.read_csv("test_YCcRUnU.csv")
df_train = df_train.drop_duplicates()
df_train = df_train[~df_train.Region_Code.isin(list(set(df_train.Region_Code)-set(df_test.Region_Code)))]
df_total = pd.concat([df_train,df_test],ignore_index=True, sort=False)
print("Shape of datasets: ", df_train.shape, df_test.shape, df_total.shape)

Shape of datasets:  (48752, 14) (21805, 13) (70557, 14)


In [247]:
df_total['holds_insurance_policy'] = np.where(df_total['Holding_Policy_Duration'].isnull(), 0,1)
df_total['Holding_Policy_Duration'].fillna(0,inplace=True) 
df_total['Holding_Policy_Duration'].replace('14+',15,inplace=True) 
df_total['Holding_Policy_Duration'] = df_total['Holding_Policy_Duration'].astype(float).astype(int)
df_total['Holding_Policy_Type'].fillna(0,inplace=True)
df_total['Health Indicator'].fillna('X0',inplace=True)
df_total['Age_diff'] = df_total['Upper_Age'] - df_total['Lower_Age']
df_total['Average_age'] = (df_total['Upper_Age'] + df_total['Lower_Age'])/2

for col in ['Health Indicator','Holding_Policy_Duration','Holding_Policy_Type',
            'Reco_Insurance_Type','Reco_Policy_Cat','holds_insurance_policy',
           'Accomodation_Type','Is_Spouse'
           ]:
    df_total['City_'+str(col)] = df_total['City_Code'].astype(str)+'_'+df_total[col].astype(str) 
    df_total['Region_'+str(col)] = df_total['Region_Code'].astype(str)+'_'+df_total[col].astype(str)

df_total['Holding_Policy_Duration_Type'] = df_total['Holding_Policy_Duration'].astype(str)+'_'+df_total['Holding_Policy_Type'].astype(str)

df_total['Reco_Insurance_Type_Cat'] = df_total['Reco_Insurance_Type'].astype(str)+'_'+df_total['Reco_Policy_Cat'].astype(str)

In [248]:
df_total['City_Code'] = df_total['City_Code'].str.replace("[^0-9]","")
df_total['City_Code'] = df_total['City_Code'].astype(float).astype(int)

df_total = pd.get_dummies(df_total,columns=['Accomodation_Type','Reco_Insurance_Type','Is_Spouse'],drop_first=True)

df_total['Health Indicator'] = df_total['Health Indicator'].str.replace("[^0-9]","")
df_total['Health Indicator'] = df_total['Health Indicator'].astype(float).astype(int)

In [249]:
df_total.columns

Index(['ID', 'City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age',
       'Health Indicator', 'Holding_Policy_Duration', 'Holding_Policy_Type',
       'Reco_Policy_Cat', 'Reco_Policy_Premium', 'Response',
       'holds_insurance_policy', 'Age_diff', 'Average_age',
       'City_Health Indicator', 'Region_Health Indicator',
       'City_Holding_Policy_Duration', 'Region_Holding_Policy_Duration',
       'City_Holding_Policy_Type', 'Region_Holding_Policy_Type',
       'City_Reco_Insurance_Type', 'Region_Reco_Insurance_Type',
       'City_Reco_Policy_Cat', 'Region_Reco_Policy_Cat',
       'City_holds_insurance_policy', 'Region_holds_insurance_policy',
       'City_Accomodation_Type', 'Region_Accomodation_Type', 'City_Is_Spouse',
       'Region_Is_Spouse', 'Holding_Policy_Duration_Type',
       'Reco_Insurance_Type_Cat', 'Accomodation_Type_Rented',
       'Reco_Insurance_Type_Joint', 'Is_Spouse_Yes'],
      dtype='object')

In [250]:
le = LabelEncoder()
for col in df_total.columns:
    if (df_total[col].dtypes == "object"):
#         df_total[col].fillna("not_available", inplace = True)
        df_total[col] = df_total[col].astype(str)
        df_total[col] = le.fit_transform(df_total[col])
#         df_total[col] = df_total[col].astype('category')

In [252]:
premium_discretizer = KBinsDiscretizer(n_bins=8, encode='ordinal', strategy='quantile')
df_total['Reco_Policy_Premium_Bins'] =premium_discretizer.fit_transform(df_total['Reco_Policy_Premium'].values.reshape(-1,1)).astype(int)

age_discretizer = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')
df_total['Upper_Age_Bins'] =age_discretizer.fit_transform(df_total['Upper_Age'].values.reshape(-1,1)).astype(int)
df_total['Lower_Age_Bins'] =age_discretizer.fit_transform(df_total['Lower_Age'].values.reshape(-1,1)).astype(int)
df_total['Age_diff_Bins'] =age_discretizer.fit_transform(df_total['Age_diff'].values.reshape(-1,1)).astype(int)
df_total['Average_age_Bins'] =age_discretizer.fit_transform(df_total['Average_age'].values.reshape(-1,1)).astype(int)

In [253]:
for col in ['City_Code','Region_Code','Upper_Age','Lower_Age','Health Indicator','Holding_Policy_Duration','Holding_Policy_Type',
           'Reco_Policy_Cat','holds_insurance_policy']:
    
    val_counts = df_total[col].value_counts().to_dict()
    df_total[str(col)+"_counts"] = df_total[col].map(val_counts)

In [254]:
for main_col in ['Reco_Policy_Premium','Upper_Age','Lower_Age','Holding_Policy_Duration',
                 
                'City_Code','Region_Code','Health Indicator','Holding_Policy_Type','Reco_Policy_Cat',
               'City_Health Indicator', 'Region_Health Indicator',
       'City_Holding_Policy_Duration', 'Region_Holding_Policy_Duration',
       'City_Holding_Policy_Type', 'Region_Holding_Policy_Type',
       'City_Reco_Insurance_Type', 'Region_Reco_Insurance_Type',
       'City_Reco_Policy_Cat', 'Region_Reco_Policy_Cat',
       'City_holds_insurance_policy', 'Region_holds_insurance_policy',
       'City_Accomodation_Type', 'Region_Accomodation_Type', 'City_Is_Spouse',
       'Region_Is_Spouse', 'Holding_Policy_Duration_Type',
       'Reco_Insurance_Type_Cat']:

    for col in ['City_Code','Region_Code','Health Indicator','Holding_Policy_Type','Reco_Policy_Cat',
               'City_Health Indicator', 'Region_Health Indicator',
       'City_Holding_Policy_Duration', 'Region_Holding_Policy_Duration',
       'City_Holding_Policy_Type', 'Region_Holding_Policy_Type',
       'City_Reco_Insurance_Type', 'Region_Reco_Insurance_Type',
       'City_Reco_Policy_Cat', 'Region_Reco_Policy_Cat',
       'City_holds_insurance_policy', 'Region_holds_insurance_policy',
       'City_Accomodation_Type', 'Region_Accomodation_Type', 'City_Is_Spouse',
       'Region_Is_Spouse', 'Holding_Policy_Duration_Type',
       'Reco_Insurance_Type_Cat']:
        
        if main_col!=col:
            df_total['unique_of_'+str(col)+'_per_'+str(main_col)] = df_total.groupby([main_col])[col].transform('nunique')


for main_col in ['City_Code','Region_Code','Health Indicator','Holding_Policy_Type','Reco_Policy_Cat',
               'City_Health Indicator', 'Region_Health Indicator',
       'City_Holding_Policy_Duration', 'Region_Holding_Policy_Duration',
       'City_Holding_Policy_Type', 'Region_Holding_Policy_Type',
       'City_Reco_Insurance_Type', 'Region_Reco_Insurance_Type',
       'City_Reco_Policy_Cat', 'Region_Reco_Policy_Cat',
       'City_holds_insurance_policy', 'Region_holds_insurance_policy',
       'City_Accomodation_Type', 'Region_Accomodation_Type', 'City_Is_Spouse',
       'Region_Is_Spouse', 'Holding_Policy_Duration_Type',
       'Reco_Insurance_Type_Cat']:
    
    for col in ['Upper_Age','Lower_Age','Holding_Policy_Duration','holds_insurance_policy',
                'Age_diff','Average_age','Reco_Policy_Premium']:  
        
        df_total['mean_of_'+str(col)+'_per_'+str(main_col)] = df_total.groupby([main_col])[col].transform('mean')
        df_total['sum_of_'+str(col)+'_per_'+str(main_col)] = df_total.groupby([main_col])[col].transform('sum')
        df_total['std_of_'+str(col)+'_per_'+str(main_col)] = df_total.groupby([main_col])[col].transform('std').fillna(-1)

In [255]:
df_total.head()

,ID,City_Code,Region_Code,Upper_Age,Lower_Age,Health Indicator,Holding_Policy_Duration,Holding_Policy_Type,Reco_Policy_Cat,Reco_Policy_Premium,Response,holds_insurance_policy,Age_diff,Average_age,City_Health Indicator,Region_Health Indicator,City_Holding_Policy_Duration,Region_Holding_Policy_Duration,City_Holding_Policy_Type,Region_Holding_Policy_Type,City_Reco_Insurance_Type,Region_Reco_Insurance_Type,City_Reco_Policy_Cat,Region_Reco_Policy_Cat,City_holds_insurance_policy,Region_holds_insurance_policy,City_Accomodation_Type,Region_Accomodation_Type,City_Is_Spouse,Region_Is_Spouse,Holding_Policy_Duration_Type,Reco_Insurance_Type_Cat,Accomodation_Type_Rented,Reco_Insurance_Type_Joint,Is_Spouse_Yes,Reco_Policy_Premium_Bins,Upper_Age_Bins,Lower_Age_Bins,Age_diff_Bins,Average_age_Bins,City_Code_counts,Region_Code_counts,Upper_Age_counts,Lower_Age_counts,Health Indicator_counts,Holding_Policy_Duration_counts,Holding_Policy_Type_counts,Reco_Policy_Cat_counts,holds_insurance_policy_counts,unique_of_City_Code_per_Reco_Policy_Premium,unique_of_Region_Code_per_Reco_Policy_Premium,unique_of_Health Indicator_per_Reco_Policy_Premium,unique_of_Holding_Policy_Type_per_Reco_Policy_Premium,unique_of_Reco_Policy_Cat_per_Reco_Policy_Premium,unique_of_City_Health Indicator_per_Reco_Policy_Premium,unique_of_Region_Health Indicator_per_Reco_Policy_Premium,unique_of_City_Holding_Policy_Duration_per_Reco_Policy_Premium,unique_of_Region_Holding_Policy_Duration_per_Reco_Policy_Premium,unique_of_City_Holding_Policy_Type_per_Reco_Policy_Premium,unique_of_Region_Holding_Policy_Type_per_Reco_Policy_Premium,unique_of_City_Reco_Insurance_Type_per_Reco_Policy_Premium,unique_of_Region_Reco_Insurance_Type_per_Reco_Policy_Premium,unique_of_City_Reco_Policy_Cat_per_Reco_Policy_Premium,unique_of_Region_Reco_Policy_Cat_per_Reco_Policy_Premium,unique_of_City_holds_insurance_policy_per_Reco_Policy_Premium,unique_of_Region_holds_insurance_policy_per_Reco_Policy_Premium,unique_of_City_Accomodation_Type_per_Reco_Policy_Premium,unique_of_Region_Accomodation_Type_per_Reco_Policy_Premium,unique_of_City_Is_Spouse_per_Reco_Policy_Premium,unique_of_Region_Is_Spouse_per_Reco_Policy_Premium,unique_of_Holding_Policy_Duration_Type_per_Reco_Policy_Premium,unique_of_Reco_Insurance_Type_Cat_per_Reco_Policy_Premium,unique_of_City_Code_per_Upper_Age,unique_of_Region_Code_per_Upper_Age,unique_of_Health Indicator_per_Upper_Age,unique_of_Holding_Policy_Type_per_Upper_Age,unique_of_Reco_Policy_Cat_per_Upper_Age,unique_of_City_Health Indicator_per_Upper_Age,unique_of_Region_Health Indicator_per_Upper_Age,unique_of_City_Holding_Policy_Duration_per_Upper_Age,unique_of_Region_Holding_Policy_Duration_per_Upper_Age,unique_of_City_Holding_Policy_Type_per_Upper_Age,unique_of_Region_Holding_Policy_Type_per_Upper_Age,unique_of_City_Reco_Insurance_Type_per_Upper_Age,unique_of_Region_Reco_Insurance_Type_per_Upper_Age,unique_of_City_Reco_Policy_Cat_per_Upper_Age,unique_of_Region_Reco_Policy_Cat_per_Upper_Age,unique_of_City_holds_insurance_policy_per_Upper_Age,unique_of_Region_holds_insurance_policy_per_Upper_Age,unique_of_City_Accomodation_Type_per_Upper_Age,unique_of_Region_Accomodation_Type_per_Upper_Age,unique_of_City_Is_Spouse_per_Upper_Age,unique_of_Region_Is_Spouse_per_Upper_Age,unique_of_Holding_Policy_Duration_Type_per_Upper_Age,unique_of_Reco_Insurance_Type_Cat_per_Upper_Age,unique_of_City_Code_per_Lower_Age,unique_of_Region_Code_per_Lower_Age,unique_of_Health Indicator_per_Lower_Age,unique_of_Holding_Policy_Type_per_Lower_Age,unique_of_Reco_Policy_Cat_per_Lower_Age,unique_of_City_Health Indicator_per_Lower_Age,unique_of_Region_Health Indicator_per_Lower_Age,unique_of_City_Holding_Policy_Duration_per_Lower_Age,unique_of_Region_Holding_Policy_Duration_per_Lower_Age,unique_of_City_Holding_Policy_Type_per_Lower_Age,unique_of_Region_Holding_Policy_Type_per_Lower_Age,unique_of_City_Reco_Insurance_Type_per_Lower_Age,unique_of_Region_Reco_Insurance_Type_per_Lower_Age,unique_of_City_Reco_Policy_Cat_p

In [256]:
df_total.columns[:35]

Index(['ID', 'City_Code', 'Region_Code', 'Upper_Age', 'Lower_Age',
       'Health Indicator', 'Holding_Policy_Duration', 'Holding_Policy_Type',
       'Reco_Policy_Cat', 'Reco_Policy_Premium', 'Response',
       'holds_insurance_policy', 'Age_diff', 'Average_age',
       'City_Health Indicator', 'Region_Health Indicator',
       'City_Holding_Policy_Duration', 'Region_Holding_Policy_Duration',
       'City_Holding_Policy_Type', 'Region_Holding_Policy_Type',
       'City_Reco_Insurance_Type', 'Region_Reco_Insurance_Type',
       'City_Reco_Policy_Cat', 'Region_Reco_Policy_Cat',
       'City_holds_insurance_policy', 'Region_holds_insurance_policy',
       'City_Accomodation_Type', 'Region_Accomodation_Type', 'City_Is_Spouse',
       'Region_Is_Spouse', 'Holding_Policy_Duration_Type',
       'Reco_Insurance_Type_Cat', 'Accomodation_Type_Rented',
       'Reco_Insurance_Type_Joint', 'Is_Spouse_Yes'],
      dtype='object')

In [258]:
# df_total.drop(['Region_Code'],axis=1,inplace=True)

In [259]:
train_x_full = df_total[df_total['Response'].notnull()]

cols = ['City_Code', 'Region_Code', 'Accomodation_Type',
       'Reco_Insurance_Type', 'Upper_Age', 'Lower_Age', 'Is_Spouse',
       'Health Indicator', 'Holding_Policy_Duration', 'Holding_Policy_Type',
       'Reco_Policy_Cat', 'Reco_Policy_Premium']

train_x_full = train_x_full[~train_x_full.loc[:,cols].duplicated(keep = 'first')].reset_index(drop=True)

test_x_full = df_total[df_total['Response'].isnull()]
train_y_full = train_x_full['Response']
test_id=test_x_full['ID']
del test_x_full['Response']
del train_x_full['Response']
del test_x_full['ID']
del train_x_full['ID']
print(train_x_full.shape, train_y_full.shape, test_x_full.shape)

(48752, 1128) (48752,) (21805, 1128)


In [260]:
test_x_full.nunique()

City_Code                                                    36
Region_Code                                                4694
Upper_Age                                                    58
Lower_Age                                                    60
Health Indicator                                             10
                                                           ... 
sum_of_Average_age_per_Reco_Insurance_Type_Cat               44
std_of_Average_age_per_Reco_Insurance_Type_Cat               44
mean_of_Reco_Policy_Premium_per_Reco_Insurance_Type_Cat      44
sum_of_Reco_Policy_Premium_per_Reco_Insurance_Type_Cat       44
std_of_Reco_Policy_Premium_per_Reco_Insurance_Type_Cat       44
Length: 1128, dtype: int64

In [261]:
for col in test_x_full.columns:
    print(col,":",len(set(train_x_full[col]).intersection(set(test_x_full[col]))))

City_Code : 36
Region_Code : 4472
Upper_Age : 58
Lower_Age : 60
Health Indicator : 10
Holding_Policy_Duration : 16
Holding_Policy_Type : 5
Reco_Policy_Cat : 22
Reco_Policy_Premium : 4486
holds_insurance_policy : 2
Age_diff : 59
Average_age : 115
City_Health Indicator : 286
Region_Health Indicator : 10148
City_Holding_Policy_Duration : 506
Region_Holding_Policy_Duration : 8425
City_Holding_Policy_Type : 174
Region_Holding_Policy_Type : 9286
City_Reco_Insurance_Type : 71
Region_Reco_Insurance_Type : 6277
City_Reco_Policy_Cat : 577
Region_Reco_Policy_Cat : 7436
City_holds_insurance_policy : 71
Region_holds_insurance_policy : 7106
City_Accomodation_Type : 71
Region_Accomodation_Type : 7070
City_Is_Spouse : 71
Region_Is_Spouse : 6010
Holding_Policy_Duration_Type : 61
Reco_Insurance_Type_Cat : 44
Accomodation_Type_Rented : 2
Reco_Insurance_Type_Joint : 2
Is_Spouse_Yes : 2
Reco_Policy_Premium_Bins : 8
Upper_Age_Bins : 10
Lower_Age_Bins : 10
Age_diff_Bins : 3
Average_age_Bins : 10
City_Code_co

unique_of_Region_Is_Spouse_per_Health Indicator : 10
unique_of_Holding_Policy_Duration_Type_per_Health Indicator : 4
unique_of_Reco_Insurance_Type_Cat_per_Health Indicator : 3
unique_of_City_Code_per_Holding_Policy_Type : 2
unique_of_Region_Code_per_Holding_Policy_Type : 5
unique_of_Health Indicator_per_Holding_Policy_Type : 1
unique_of_Reco_Policy_Cat_per_Holding_Policy_Type : 1
unique_of_City_Health Indicator_per_Holding_Policy_Type : 5
unique_of_Region_Health Indicator_per_Holding_Policy_Type : 5
unique_of_City_Holding_Policy_Duration_per_Holding_Policy_Type : 5
unique_of_Region_Holding_Policy_Duration_per_Holding_Policy_Type : 5
unique_of_City_Holding_Policy_Type_per_Holding_Policy_Type : 2
unique_of_Region_Holding_Policy_Type_per_Holding_Policy_Type : 5
unique_of_City_Reco_Insurance_Type_per_Holding_Policy_Type : 4
unique_of_Region_Reco_Insurance_Type_per_Holding_Policy_Type : 5
unique_of_City_Reco_Policy_Cat_per_Holding_Policy_Type : 5
unique_of_Region_Reco_Policy_Cat_per_Holding

unique_of_Region_holds_insurance_policy_per_City_Holding_Policy_Type : 118
unique_of_City_Accomodation_Type_per_City_Holding_Policy_Type : 2
unique_of_Region_Accomodation_Type_per_City_Holding_Policy_Type : 128
unique_of_City_Is_Spouse_per_City_Holding_Policy_Type : 2
unique_of_Region_Is_Spouse_per_City_Holding_Policy_Type : 127
unique_of_Holding_Policy_Duration_Type_per_City_Holding_Policy_Type : 15
unique_of_Reco_Insurance_Type_Cat_per_City_Holding_Policy_Type : 42
unique_of_City_Code_per_Region_Holding_Policy_Type : 1
unique_of_Region_Code_per_Region_Holding_Policy_Type : 1
unique_of_Health Indicator_per_Region_Holding_Policy_Type : 8
unique_of_Holding_Policy_Type_per_Region_Holding_Policy_Type : 1
unique_of_Reco_Policy_Cat_per_Region_Holding_Policy_Type : 12
unique_of_City_Health Indicator_per_Region_Holding_Policy_Type : 8
unique_of_Region_Health Indicator_per_Region_Holding_Policy_Type : 8
unique_of_City_Holding_Policy_Duration_per_Region_Holding_Policy_Type : 14
unique_of_Region

unique_of_City_Holding_Policy_Type_per_Region_holds_insurance_policy : 4
unique_of_Region_Holding_Policy_Type_per_Region_holds_insurance_policy : 4
unique_of_City_Reco_Insurance_Type_per_Region_holds_insurance_policy : 2
unique_of_Region_Reco_Insurance_Type_per_Region_holds_insurance_policy : 2
unique_of_City_Reco_Policy_Cat_per_Region_holds_insurance_policy : 15
unique_of_Region_Reco_Policy_Cat_per_Region_holds_insurance_policy : 15
unique_of_City_holds_insurance_policy_per_Region_holds_insurance_policy : 1
unique_of_City_Accomodation_Type_per_Region_holds_insurance_policy : 2
unique_of_Region_Accomodation_Type_per_Region_holds_insurance_policy : 2
unique_of_City_Is_Spouse_per_Region_holds_insurance_policy : 2
unique_of_Region_Is_Spouse_per_Region_holds_insurance_policy : 2
unique_of_Holding_Policy_Duration_Type_per_Region_holds_insurance_policy : 34
unique_of_Reco_Insurance_Type_Cat_per_Region_holds_insurance_policy : 19
unique_of_City_Code_per_City_Accomodation_Type : 1
unique_of_Re

mean_of_Upper_Age_per_City_Code : 36
sum_of_Upper_Age_per_City_Code : 36
std_of_Upper_Age_per_City_Code : 36
mean_of_Lower_Age_per_City_Code : 36
sum_of_Lower_Age_per_City_Code : 36
std_of_Lower_Age_per_City_Code : 36
mean_of_Holding_Policy_Duration_per_City_Code : 36
sum_of_Holding_Policy_Duration_per_City_Code : 36
std_of_Holding_Policy_Duration_per_City_Code : 36
mean_of_holds_insurance_policy_per_City_Code : 36
sum_of_holds_insurance_policy_per_City_Code : 36
std_of_holds_insurance_policy_per_City_Code : 36
mean_of_Age_diff_per_City_Code : 36
sum_of_Age_diff_per_City_Code : 36
std_of_Age_diff_per_City_Code : 36
mean_of_Average_age_per_City_Code : 36
sum_of_Average_age_per_City_Code : 36
std_of_Average_age_per_City_Code : 36
mean_of_Reco_Policy_Premium_per_City_Code : 36
sum_of_Reco_Policy_Premium_per_City_Code : 36
std_of_Reco_Policy_Premium_per_City_Code : 36
mean_of_Upper_Age_per_Region_Code : 2402
sum_of_Upper_Age_per_Region_Code : 1559
std_of_Upper_Age_per_Region_Code : 4292
me

sum_of_Reco_Policy_Premium_per_City_Holding_Policy_Duration : 506
std_of_Reco_Policy_Premium_per_City_Holding_Policy_Duration : 507
mean_of_Upper_Age_per_Region_Holding_Policy_Duration : 1439
sum_of_Upper_Age_per_Region_Holding_Policy_Duration : 777
std_of_Upper_Age_per_Region_Holding_Policy_Duration : 4431
mean_of_Lower_Age_per_Region_Holding_Policy_Duration : 1411
sum_of_Lower_Age_per_Region_Holding_Policy_Duration : 747
std_of_Lower_Age_per_Region_Holding_Policy_Duration : 4514
mean_of_Holding_Policy_Duration_per_Region_Holding_Policy_Duration : 16
sum_of_Holding_Policy_Duration_per_Region_Holding_Policy_Duration : 56
std_of_Holding_Policy_Duration_per_Region_Holding_Policy_Duration : 2
mean_of_holds_insurance_policy_per_Region_Holding_Policy_Duration : 2
sum_of_holds_insurance_policy_per_Region_Holding_Policy_Duration : 21
std_of_holds_insurance_policy_per_Region_Holding_Policy_Duration : 2
mean_of_Age_diff_per_Region_Holding_Policy_Duration : 640
sum_of_Age_diff_per_Region_Holding

sum_of_holds_insurance_policy_per_City_holds_insurance_policy : 36
std_of_holds_insurance_policy_per_City_holds_insurance_policy : 1
mean_of_Age_diff_per_City_holds_insurance_policy : 71
sum_of_Age_diff_per_City_holds_insurance_policy : 71
std_of_Age_diff_per_City_holds_insurance_policy : 71
mean_of_Average_age_per_City_holds_insurance_policy : 71
sum_of_Average_age_per_City_holds_insurance_policy : 71
std_of_Average_age_per_City_holds_insurance_policy : 71
mean_of_Reco_Policy_Premium_per_City_holds_insurance_policy : 71
sum_of_Reco_Policy_Premium_per_City_holds_insurance_policy : 71
std_of_Reco_Policy_Premium_per_City_holds_insurance_policy : 71
mean_of_Upper_Age_per_Region_holds_insurance_policy : 2303
sum_of_Upper_Age_per_Region_holds_insurance_policy : 1232
std_of_Upper_Age_per_Region_holds_insurance_policy : 6149
mean_of_Lower_Age_per_Region_holds_insurance_policy : 2240
sum_of_Lower_Age_per_Region_holds_insurance_policy : 1169
std_of_Lower_Age_per_Region_holds_insurance_policy : 

### Catboost

In [266]:
for col in ['City_Code', 'Region_Code', 
               'Health Indicator', 'Holding_Policy_Type',
               'Reco_Policy_Cat','holds_insurance_policy', 
               'City_Health Indicator', 'Region_Health Indicator',
               'City_Holding_Policy_Type', 'Region_Holding_Policy_Type',
               'City_Reco_Insurance_Type', 'Region_Reco_Insurance_Type',
               'City_Reco_Policy_Cat', 'Region_Reco_Policy_Cat',
               'City_holds_insurance_policy', 'Region_holds_insurance_policy',
               'City_Accomodation_Type', 'Region_Accomodation_Type', 'City_Is_Spouse',
               'Region_Is_Spouse', 'Holding_Policy_Duration_Type',
               'Reco_Insurance_Type_Cat', 'Accomodation_Type_Rented',
               'Reco_Insurance_Type_Joint', 'Is_Spouse_Yes',
               'City_Holding_Policy_Duration', 'Region_Holding_Policy_Duration',
               'Holding_Policy_Duration','Reco_Policy_Premium',
               'Upper_Age', 'Lower_Age','Age_diff',
               'Accomodation_Type_Rented','Reco_Insurance_Type_Joint', 'Is_Spouse_Yes'
              ]:
    train_x_full[col] = train_x_full[col].astype(int)
    test_x_full[col] = test_x_full[col].astype(int)

cat_features = ['City_Code', 'Region_Code', 
               'Health Indicator', 'Holding_Policy_Type',
               'Reco_Policy_Cat','holds_insurance_policy', 
               'City_Health Indicator', 'Region_Health Indicator',
               'City_Holding_Policy_Type', 'Region_Holding_Policy_Type',
               'City_Reco_Insurance_Type', 'Region_Reco_Insurance_Type',
               'City_Reco_Policy_Cat', 'Region_Reco_Policy_Cat',
               'City_holds_insurance_policy', 'Region_holds_insurance_policy',
               'City_Accomodation_Type', 'Region_Accomodation_Type', 'City_Is_Spouse',
               'Region_Is_Spouse', 'Holding_Policy_Duration_Type',
               'Reco_Insurance_Type_Cat', 'Accomodation_Type_Rented',
               'Reco_Insurance_Type_Joint', 'Is_Spouse_Yes',
               'City_Holding_Policy_Duration', 'Region_Holding_Policy_Duration',
               'Holding_Policy_Duration','Reco_Policy_Premium',
               'Accomodation_Type_Rented','Reco_Insurance_Type_Joint', 'Is_Spouse_Yes'
              ]

In [268]:
# Final ROC-AUC = 0.8139
train_x = train_x_full.copy()
train_y = train_y_full.copy()
test_x = test_x_full.copy()
feature_importance_df = pd.DataFrame()

NFOLDS = 20
folds = StratifiedKFold(n_splits=NFOLDS, shuffle=True, random_state=18121995)
pred_test = []
pred_x = []
pred_y = []
j=1
for fold, (train_ids, test_ids) in enumerate(folds.split(train_x, train_y)):
    print('● Fold :', fold+1)
    model = CatBoostClassifier(n_estimators=20000,random_state=1812195,learning_rate=0.03,eval_metric='AUC',
                              cat_features =cat_features)
    model.fit(train_x.loc[train_ids], train_y.loc[train_ids], 
              eval_set=[(train_x.loc[train_ids], train_y.loc[train_ids]), (train_x.loc[test_ids], train_y.loc[test_ids])],
              verbose=500,
              early_stopping_rounds=200)
    pred_fold = model.predict_proba(train_x.loc[test_ids])[:,-1]
    pred_x.extend([float(i) for i  in pred_fold])
    pred_y.extend(list(train_y.loc[test_ids].values))
    pred_fold_test = model.predict_proba(test_x)[:,-1]
    pred_test.append([float(i) for i  in pred_fold_test])
    print('\n')
    
    fold_importance_df = pd.DataFrame()
    fold_importance_df["feature"] = train_x.loc[train_ids].columns
    fold_importance_df["importance"] = model.feature_importances_
    fold_importance_df["fold"] = j + 1
    feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
    j=j+1

print("Final ROC-AUC Score:", roc_auc_score(pred_y, pred_x))

final_prediction = pd.DataFrame(pred_test).T
final_prediction.columns = [("FOLD_"+str(i)) for i in range(final_prediction.shape[1])]

vote = final_prediction.mean(axis=1)
final_prediction['Response'] = vote
final_prediction['ID'] = test_id.values
print(final_prediction.shape)

train_pred_all['cb_pred'] = pred_x
final_prediction_cb = final_prediction.copy()

final_prediction.head()

● Fold : 1
0:	test: 0.7499461	test1: 0.7449343	best: 0.7449343 (0)	total: 316ms	remaining: 1h 45m 18s
500:	test: 0.8859171	test1: 0.8069502	best: 0.8071494 (467)	total: 2m 32s	remaining: 1h 39m
1000:	test: 0.9023302	test1: 0.8076679	best: 0.8081780 (841)	total: 5m 6s	remaining: 1h 36m 49s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.8081780066
bestIteration = 841

Shrink model to first 842 iterations.


● Fold : 2
0:	test: 0.7491410	test1: 0.7610509	best: 0.7610509 (0)	total: 329ms	remaining: 1h 49m 31s
500:	test: 0.8882595	test1: 0.8235128	best: 0.8237693 (472)	total: 2m 49s	remaining: 1h 49m 55s
Stopped by overfitting detector  (200 iterations wait)

bestTest = 0.823847676
bestIteration = 543

Shrink model to first 544 iterations.


● Fold : 3
0:	test: 0.7404455	test1: 0.7338029	best: 0.7338029 (0)	total: 331ms	remaining: 1h 50m 10s
500:	test: 0.8882716	test1: 0.8096393	best: 0.8097011 (494)	total: 2m 48s	remaining: 1h 49m 14s
Stopped by overfitting detector  

,FOLD_0,FOLD_1,FOLD_2,FOLD_3,FOLD_4,FOLD_5,FOLD_6,FOLD_7,FOLD_8,FOLD_9,FOLD_10,FOLD_11,FOLD_12,FOLD_13,FOLD_14,FOLD_15,FOLD_16,FOLD_17,FOLD_18,FOLD_19,Response,ID
0,0.064973,0.068683,0.059546,0.068678,0.063358,0.054752,0.072979,0.052199,0.067585,0.067355,0.056773,0.064627,0.054046,0.053868,0.059736,0.048528,0.066159,0.067880,0.055920,0.052007,0.060983,50883
1,0.067165,0.078078,0.084768,0.070015,0.079540,0.063310,0.086575,0.059024,0.083825,0.071201,0.084387,0.076651,0.073158,0.075502,0.084388,0.082494,0.076802,0.052200,0.073131,0.072769,0.074749,50884
2,0.434917,0.457602,0.405649,0.463660,0.439468,0.427119,0.482638,0.451075,0.451873,0.457026,0.449572,0.462557,0.470639,0.423898,0.453164,0.433602,0.420445,0.553237,0.463387,0.449850,0.452569,50885
3,0.091021,0.103002,0.099288,0.089482,0.098358,0.097389,0.095846,0.093021,0.103143,0.094719,0.094996,0.101650,0.101842,0.084432,0.096496,0.092977,0.093867,0.093417,0.085816,0.089696,0.095023,50886
4,0.038729,0.042363,0.031359,0.034737,0.035625,0.034278,0.031901,0.041032,0.036576,0.034473,0.036095,0.039388,0.034034,0.038677,0.037061,0.036921,0.042954,0.025112,0.039901,0.038533,0.036487,50887


In [269]:
df_submission = final_prediction[['ID', 'Response']]
df_submission.to_csv("cb_v4.csv", index = False)
df_submission.head()

,ID,Response
0,50883,0.060983
1,50884,0.074749
2,50885,0.452569
3,50886,0.095023
4,50887,0.036487


In [270]:
all_features = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)
all_features.reset_index(inplace=True)
important_features = list(all_features[all_features['importance']!=0]['feature'])
all_features

,feature,importance
0,Region_Reco_Policy_Cat,4.847963
1,sum_of_Lower_Age_per_Region_Reco_Policy_Cat,4.382607
2,City_Reco_Policy_Cat,4.031894
3,sum_of_Average_age_per_Region_Reco_Policy_Cat,3.575629
4,sum_of_Upper_Age_per_Region_Reco_Policy_Cat,3.455060
...,...,...
1123,unique_of_Reco_Policy_Cat_per_Upper_Age,0.000000
1124,unique_of_City_Code_per_City_Reco_Insurance_Type,0.000000
1125,unique_of_City_Code_per_City_Reco_Policy_Cat,0.000000
1126,unique_of_City_Code_per_City_holds_insurance_p...,0.000000
